In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import MELODIC
%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [3]:
exp = 'ser'
sub = 'fd_104'
run = str(1)
sub_dir = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + run + '/'
in_file = ''
out_dir = sub_dir + 'aroma/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
affmat = sub_dir + 'melodic/reg/highres2example_func.mat'
warp = home_dir + 'data/' + sub + '/normalization/brain_warp.nii.gz'
mc = sub_dir + 'melodic/mc/prefiltered_func_data_mcf.par'
mask = sub_dir + 'functional_mask.nii.gz'

aroma_f = '/home/iballard/ICA-AROMA-master/ICA_AROMA.py'
cmd_str = [aroma_f,'-in',in_file,'-out',out_dir,'-mc',
           mc,'-m',mask,'-affmat',affmat,'-warp',warp]
print ' '.join(cmd_str)

/home/iballard/ICA-AROMA-master/ICA_AROMA.py -in  -out /data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/aroma/ -mc /data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/melodic/mc/prefiltered_func_data_mcf.par -m /data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/functional_mask.nii.gz -affmat /data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/melodic/reg/highres2example_func.mat -warp /data/home/iballard/fd/data/fd_104/normalization/brain_warp.nii.gz


In [54]:
files = ['artifact_detection.png', 'realignment_plots.png', 'func2anat.png',
         'func2anat_tkreg.dat', 'func2anat_flirt.mat', 'functional_mask.png', 
         'mean_func.png', 'example_func.png', 'example_func.nii.gz', 'realignment_params.csv', 
         'artifacts.csv', 'functional_mask.nii.gz', 'mean_func.nii.gz', 'experiment_info.json']
exp = 'sim'
new_dir = home_dir + 'analysis/' + exp + '_4mm/'

for s in sub_list:
    new_s_dir = new_dir + s
    if not os.path.exists(new_s_dir):
        os.mkdir(new_s_dir)
    new_s_dir = new_dir + s + '/preproc'
    if not os.path.exists(new_s_dir):
        os.mkdir(new_s_dir)
        
    for run in range(1,4):
        old = home_dir + 'analysis/' + exp + '_8mm/' + s + '/preproc/run_' + str(run) + '/'
        new = home_dir + 'analysis/' + exp + '_4mm/' + s + '/preproc/run_' + str(run) + '/'
        if not os.path.exists(new):
            os.mkdir(new)
        for f in files:
            o = old + f
            n = new+f
            if not os.path.exists(n):
                cmd = ['ln','-s',o,n]
#                 os.system(' '.join(cmd))
        
        smooth_clean = home_dir + '/smoothed/smoothed' + str(run) + exp + s + '/_smooth0/' + \
        'filtered_func_data_clean_smooth.nii.gz'
        new_smooth = new + 'smoothed_timeseries.nii.gz'
        if not os.path.exists(new_smooth) and os.path.exists(smooth_clean):
            cmd = ['mv',smooth_clean,new_smooth]
#             print cmd
            os.system(' '.join(cmd))

#         if os.path.exists(new_smooth):
            
#             try:
#                 a = nib.load(new_smooth)
#                 orig = nib.load(old + 'smoothed_timeseries.nii.gz')
#                 if a.shape[3] != orig.shape[3]:
#                     print new_smooth
#             except:
#                 print new_smooth